<a href="https://colab.research.google.com/github/ITU-Business-Analytics-Team/Business_Analytics_for_Professionals/blob/main/Part%20I%20%3A%20Methods%20%26%20Technologies%20for%20Business%20Analytics/Chapter%207%3A%20Text%20Analytics/7_6_3_Deep_Learning_Based_Sentiment_Analysis__XLNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis (Opinion Mining)**
## Deep Learning Based Sentiment Analysis

The sentiment analysis of commodity news task are previously investigated under statistical methods. Since it is also mentioned at the end of that notebook, sentiment analysis with a few instances (there are only 1120 news total for train and test) is a complex problem. In the latest years, research on NLP models advanced and produced some high quality classification models, most of them are deep learning based methods. In this notebook, the XLNet approach will be introduced.



### XLNET

XLNET is another transformers architecture based sequence classification algorithm. Its implementation and results very similar to BERT. PyTorch will be used again and even most of the previously defined functions in BERT notebook will be reused.

In [ ]:
!pip install sentencepiece
# restart the runtime from runtime tab in colab toolbar if you download the sentencepiece for first time

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
!pip install transformers
!pip install torch torchvision torchaudio

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 3.3 MB 33.5 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 596 kB 39.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 1.9 MB 17.7 MB/s 
     |████████████████████████████████| 1.9 MB 30.9 MB/s 


In [ ]:
# for deep learning implementation
import torch
# to work on the dataset
import pandas as pd
# to follow progress as bar in notebook
from tqdm.notebook import tqdm

In [ ]:
# read the data  
url=   'https://docs.google.com/spreadsheets/d/1XXyxrd7r0mx7kyLaYHDVwh6BFJzo8cPD/edit?usp=sharing&ouid=108589602591644119588&rtpof=true&sd=true'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

df = pd.read_excel(path)
df['summary'] = df['summary'].map(lambda x: x.lstrip('News :'))
df['summary'] = df['summary'].map(lambda x: x.lstrip('UPDATE'))
df['summary'] = df['summary'].map(lambda x: x.lstrip('METALS-'))
df.rename(columns={'summary':'text'}, inplace = True)

In [ ]:
url=   'https://docs.google.com/spreadsheets/d/145tqf2J949KGCYnH-Nx3hiaHTogiZFn4/edit?usp=sharing&ouid=108589602591644119588&rtpof=true&sd=true'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
test_df = pd.read_excel(path)
test_df['summary'] = test_df['summary'].map(lambda x: x.lstrip('News :'))
test_df['summary'] = test_df['summary'].map(lambda x: x.lstrip('UPDATE'))
test_df['summary'] = test_df['summary'].map(lambda x: x.lstrip('METALS-'))
test_df.rename(columns={'summary':'text'}, inplace = True)
test_df

,text,sentiment
0,Copper at near 2-week highs on hopes China imp...,0
1,"China's Yunnan to help firms stockpile 110,000...",1
2,COLUMN-Politics trumps aluminium as U.S. reimp...,-1
3,Base metals decline on weak China demand outlook,-1
4,"ALUMINIUM FALLS TO $1,751.50/T, LOWEST SINCE...",-1
...,...,...
163,China names former Chinalco exec as industry m...,1
164,Copper edges off two-year low as Washington so...,0
165,"Uncertainty on global growth, trade war weighs...",-1
166,Copper gains after Fed chief rekindles rate cu...,1


In [ ]:
df = df.drop_duplicates().merge(test_df.drop_duplicates(), on=test_df.columns.to_list(), 
                   how='left', indicator=True, right_index = False, left_index = False)
df = df.loc[df._merge=='left_only',df.columns!='_merge']
df = df.reset_index(drop = True, inplace= False)

In [ ]:
df.sentiment.value_counts()

 1    486
-1    366
 0     64
Name: sentiment, dtype: int64

In [ ]:
possible_labels = df.sentiment.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
df['label'] = df.sentiment.replace(label_dict)
test_df['label'] = test_df.sentiment.replace(label_dict)

In [ ]:
df['data_type'] = ['train']*df.shape[0]
test_df['data_type'] = ['val']*test_df.shape[0]

In [ ]:
df.groupby(['sentiment', 'label', 'data_type']).count()

,,,text
sentiment,label,data_type,
-1,1,train,366
0,2,train,64
1,0,train,486


In [ ]:
test_df.groupby(['sentiment', 'label', 'data_type']).count()

,,,text
sentiment,label,data_type,
-1,1,val,65
0,2,val,12
1,0,val,91


In [ ]:
from torch.utils.data import TensorDataset
from transformers import XLNetTokenizer, XLNetModel

As different than BERT, 'xlnet-large-cased' tokenizer will be used.

In [ ]:
PRE_TRAINED_MODEL_NAME = 'xlnet-large-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/761 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(    df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=52, 
    return_tensors='pt')

encoded_data_val = tokenizer.batch_encode_plus(
    test_df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=52, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test_df.label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels, test=False):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    overall_acc = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        acc = (len(y_preds[y_preds==label])/len(y_true))*100
        overall_acc += acc * len(y_preds)
        print(f'Accuracy: {acc}\n')
    if (test==False):
        print(f'Overall Accuracy: {overall_acc/len(dataset_val)}\n')
    else:
        print(f'Overall Accuracy: {overall_acc/len(dataset_test)}\n')
    

In [ ]:
from transformers import XLNetForSequenceClassification

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=len(label_dict), mem_len=1024)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=5e-5, 
                  eps=1e-8)

In [ ]:
epochs = 25

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_XLNET_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/29 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.930281425344533
Validation loss: 0.9117169777552286
F1 Score (Weighted): 0.3806306306306307


Epoch 2:   0%|          | 0/29 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.9007037836929848
Validation loss: 0.8955048223336538
F1 Score (Weighted): 0.3806306306306307


...


Epoch 23
Training loss: 0.0437426020547844
Validation loss: 2.020647943019867
F1 Score (Weighted): 0.6714736441024497


Epoch 24:   0%|          | 0/29 [00:00<?, ?it/s]


Epoch 24
Training loss: 0.035648891668172616
Validation loss: 2.095287561416626
F1 Score (Weighted): 0.67214858580248


Epoch 25:   0%|          | 0/29 [00:00<?, ?it/s]


Epoch 25
Training loss: 0.033835522214287955
Validation loss: 2.073530375957489
F1 Score (Weighted): 0.6730248299743503


>The performance of the model could deviate from one runtime to other due to randomization in optimization algorithms which ensure to reach global optimal optimum point. In order to reduce the effect of randomization, the model could run several times and the results could be averaged. You can go ahead and try this approach. \\
>Alternatively, a pretrained well-performed model is added here. The following cells can be used to download and use it. PyTorch will import the weights and bias from the given pretrained model.

In [ ]:
!pip install gdown

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1tftMIqAUw66hRwS8xCIoAIU6TKNOFqYF'
output = 'finetuned_XLNET_epoch_19.model'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1tftMIqAUw66hRwS8xCIoAIU6TKNOFqYF
To: /content/finetuned_XLNET_epoch_19.model
100%|██████████| 469M/469M [00:03<00:00, 127MB/s]


'finetuned_XLNET_epoch_19.model'

In [ ]:
#if your colab runtime machine has cpu please uncomment the line below
model.load_state_dict(torch.load('finetuned_XLNET_epoch_19.model',map_location=torch.device('cpu')))

# if your colab runtime machine has gpu please use this line
#model.load_state_dict(torch.load('finetuned_XLNET_epoch_19.model'), predictions, true_vals = evaluate(dataloader_validation))

<All keys matched successfully>

In [ ]:
f1_score_func(predictions, true_vals)

0.6730248299743503

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 1
Accuracy: 82.41758241758241

Class: -1
Accuracy: 61.53846153846154

Class: 0
Accuracy: 8.333333333333332

Overall Accuracy: 69.04761904761905



As mentioned before, the result is very similar to Bert's performance and outperforms statistical models. The model produces more accurate predictions on positive news as expected due to imbalanced dataset. 